**Projeto 2 | Modelagem e Simulação do Mundo Físico** 1B Engenharia - Lucas Fontenla & Victor Hugo Leal

Em dias chuvosos (nublados), os passageiros que utilizam transporte público fecham as janelas para evitar a entrada de água no veículo, porém, conforme o horário do rush se aproxima, mais pessoas entram no ônibus e a temperatura interna do ônibus aumenta muito. 

1. Como o número de pessoas afeta a temperatura interna do ônibus?
2. Para que uma determinada empresa aumente o conforto dos passageiros, qual ar condicionado ela deverá instalar no ônibus?

**O modelo:**

<img src="Modelo.jpg">

Fluxos de calor 
1. Qpp - Emitido pelas pessoas de pé
2. Qps - Emitido pelas pessoas sentadas
3. Qac - Absorvido pelo ar condicionado
4. Qp - Que sai pela porta
5. Qj - Que sai pela janela
6. Ql - Que sai pela lataria

Variáveis Exógenas
1. np - Número de pessoas de pé
2. ns - Número de pessoas sentadas
3. t - Tempo de abertura da porta
4. Te - Temperatura externa

**Equação do Modelo:**

<img src="Equação Diferencial.png", width="330", height="56">


**Perda de calor por CONDUÇÃO**

O ar interno fornece calor para o vidro e para a lataria. O fluxo de calor no interior do material se dá por condução

Resistência térmica de condução:

<img src="Rcond.png", width="122", height="45">

1. L: Espessura do material [m]
2. h: Coeficiente de condutividade térmica [W/mK]
3. As: Área superficial [m^2]

Fluxo de calor:

<img src="Qcond.png", width="112", height="40">

1. DeltaT - (Temp. Interna - Temp. Externa)


**Perda de calor por CONVECÇÃO**

O vidro, a lataria e quando a porta se abre, o ar interno, trocam calor com o meio externo

Resistência térmica de convecção:

<img src="Rconv.png", width="122", height="45">

1. h: Coeficiente de condutividade térmica [W/mK]
2. As: Área superficial [m^2]

Fluxo de calor:

<img src="Qconv.png", width="112", height="40">

1. DeltaT - (Temp. Interna - Temp. Externa)

**EQUAÇÕES FINAIS**
<img src="Qj.png">
<img src="Ql.png">
<img src="Qp.png">

In [80]:
"""Função que calcula Qj
nj (número de janelas) = 20
hj (altura da janela) = 1.2
lj (largura da janela) = 0.9 
hp (altura do para-brisa) = 2.6
lp (largura do para-brisa) = 1.7
hvidro = 0.8
har = 0.023
Lvidro (média) = 3mm = 3*10**-3m"""

def Rcond_vidros(Area_vidro, hvidro, Lvidro):
    return Lvidro/(hvidro*Area_vidro)

def Rconv_vidros(Area_vidro, hvidro, har):
    return 1/(har*Area_vidro)

def Qvidro(Tinterna, Texterna, hvidro, Lvidro, har, Area_vidro):
    return (Tinterna - Texterna)/(Rcond_vidros(Area_vidro, hvidro, Lvidro ) + Rconv_vidros(Area_vidro, hvidro, har))

In [81]:
"""Função que calcula Ql
ll (largura da lataria) = 2.6
hl (altura da lataria) = 3.5
cl (comprimento da lataria) = 18.6
haço = 52
Laço = 3mm = 3*10**-3m"""

def Rcond_lataria(haco, Laco, Area_lataria):
    return Laco/(haco*Area_lataria)

def Rconv_lataria(har, Area_lataria):
    return 1/(har*Area_lataria)

def QLataria(Tinterna, Texterna, haco, Laco, har):
    return (Tinterna - Texterna)/(Rcond_lataria(Area_lataria, haco, Laco) + Rconv_lataria(har, Area_lataria))

In [82]:
"""Função que calcula Qp
np (número de portas) = 3
ap (altura da porta) = 1.9
Lp (largura da porta) = 1.1"""

#def AreaTotal_porta(numeor_porta, altura_porta, largura_porta):
 #   return numero_porta*altura_porta*largura_porta

def Rconv_porta(har, Area_porta):
    return 1/(har*Area_porta)

def Qporta(Tinterna, Texterna, Area_porta, har):
    return (Tinterna - Texterna)/(Rconv_porta(Area_porta, har))

In [83]:
"""Função que calcula Q_pessoas_pe
numero_pe = 2 (inicial)
Qppp (por pessoa de pé) = 167W"""

def Q_pessoas_pe(numero_pe, Qpp):
    return numero_pe*Qppp

In [84]:
"""Função que calcula Qps
numero_ps = 0 (inicial)
Qpps (por pessoa sentada) = 115W"""

def Qps(numero_ps, Qpps):
    return numero_ps*Qpps

In [85]:
"""Definição dos Parâmetros"""

hvidro = 0.8 #condutividade térmica do vidro
har = 0.023 #condutividade térmica do ar
haço = 52 #condutividade térmica do aço

Lvidro = 3*10**-3 #espessura do aço
Laço = 3*10**-3 #espessura da lataria

Area_lataria = 206.56 #área d lataria em contato com o ar
Area_janela = 21.6 #área da janela em contato com o ar
Area_parabrisa = 4.42 #área do para-brisa
Area_porta = 6.27 #área do ar interno em contato com o ar externo quando a porta aberta


Area_vidro = Area_janela + Area_parabrisa + Area_porta #A área em vidro inicial conta como se a porta fosse de vidro
